In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchsummary import summary
from fvcore.nn import FlopCountAnalysis

In [2]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# CIFAR-10 数据集的转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图片大小以匹配 ViT 输入
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载 CIFAR-10 数据集
train_dataset = datasets.CIFAR10(root='~/.cache', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root='~/.cache', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# 加载预训练的 Vision Transformer 模型
model = models.vit_b_32(pretrained=True)
model.heads = nn.Linear(model.heads[0].in_features, 10)  # 调整分类器头以适应 CIFAR-10
model = model.to(device)
print(model)

c:\Users\jimaz\anaconda3\envs\python38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jimaz\anaconda3\envs\python38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_32_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_32_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [5]:
summary(model, (3, 224, 224))
flops = FlopCountAnalysis(model, torch.randn(1, 3, 224, 224).to(device))
print("FLOPs: {:.2f}GFLOPS".format(flops.total() / 1e9))

Layer (type:depth-idx)                        Output Shape              Param #
├─Conv2d: 1-1                                 [-1, 768, 7, 7]           2,360,064
├─Encoder: 1-2                                [-1, 50, 768]             --
|    └─Dropout: 2-1                           [-1, 50, 768]             --
|    └─Sequential: 2-2                        [-1, 50, 768]             --
|    |    └─EncoderBlock: 3-1                 [-1, 50, 768]             7,087,872
|    |    └─EncoderBlock: 3-2                 [-1, 50, 768]             7,087,872
|    |    └─EncoderBlock: 3-3                 [-1, 50, 768]             7,087,872
|    |    └─EncoderBlock: 3-4                 [-1, 50, 768]             7,087,872
|    |    └─EncoderBlock: 3-5                 [-1, 50, 768]             7,087,872
|    |    └─EncoderBlock: 3-6                 [-1, 50, 768]             7,087,872
|    |    └─EncoderBlock: 3-7                 [-1, 50, 768]             7,087,872
|    |    └─EncoderBlock: 3-8          

Unsupported operator aten::mul encountered 49 time(s)
Unsupported operator aten::add encountered 25 time(s)
Unsupported operator aten::div encountered 12 time(s)
Unsupported operator aten::unflatten encountered 12 time(s)
Unsupported operator aten::scaled_dot_product_attention encountered 12 time(s)
Unsupported operator aten::gelu encountered 12 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
encoder.layers.encoder_layer_0.self_attention.out_proj, encoder.layers.encoder_layer_1.self_attention.out_proj, encoder.layers.encoder_layer_10.self_attention.out_proj, encoder.layers.encoder_layer_11.self_attention.out_proj, encoder.layers.encoder_layer_2.self_attention.out_proj, encoder.layers.encoder_layer_3.self_atte

FLOPs: 4.37GFLOPS


In [6]:
# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练循环
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit='batch') as tepoch:
        for images, labels in tepoch:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            tepoch.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

    # 模型评估
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

Epoch 1/3:   0%|          | 0/782 [00:00<?, ?batch/s]

Epoch 1/3:   7%|▋         | 58/782 [00:12<02:38,  4.58batch/s, loss=2.26]


KeyboardInterrupt: 

In [9]:
# 推理速度测试
import time
inputs = torch.randn(1, 3, 224, 224).to(device)
model.eval()
start = time.time()
with torch.no_grad():
    for _ in range(1000):
        outputs = model(inputs)
end = time.time()
print("Inference time: {:.2f}ms".format((end - start) * 1000 / 1000))

Inference time: 4.56ms
